In [4]:
import os
print(os.getcwd())
print(os.listdir())

c:\Users\varun\Box Sync\Python\Python Projects\Yoda_Agent
['Additional_Scripts', 'archive (1)', 'dialogue_output.csv', 'local_llm.py', 'Official Yoda Prequel Dialouge', 'Official Yoda Prequel Dialouge - Context', 'script_agent.py', 'STAR WARS THE ATTACK OF THE CLONES.txt', 'STAR WARS THE PHANTOM MENACE.txt', 'STAR WARS THE REVENGE OF THE SITH.txt', 'star_wars_dialogue.csv', 'SW_EpisodeIV.txt', 'SW_EpisodeV.txt', 'SW_EpisodeVI.txt', 'TPM_Dialogue_Complete.xlsx', 'yoda_all.csv', 'YODA_AOTC - Copy.txt', 'YODA_AOTC.txt', 'yoda_chat.jsonl', 'yoda_chat_01.jsonl', 'yoda_chat_system.jsonl', 'yoda_data.ipynb', 'yoda_data_01.ipynb', 'yoda_dialogue.csv', 'yoda_model', 'Yoda_Model.ipynb', 'yoda_model_test.py', 'YODA_ROTS.txt', 'YODA_TPM - Copy.txt', 'YODA_TPM.txt']


Below code fixes the json format into a proper JSONL format. The output says yoda_chat_final, however I renamed it to yoda_chat_02 and I renamed it in windows explorer and in this code block as well.

In [ ]:
import json
import re

def decode_unicode_escapes(text):
    """Decode Unicode escape sequences in text"""
    if isinstance(text, str):
        # Decode \uXXXX sequences
        return text.encode().decode('unicode_escape')
    return text

def fix_jsonl_with_unicode_decoding(input_file, output_file):
    with open(input_file, "r", encoding='utf-8') as f:
        content = f.read()
    
    # First decode any Unicode escapes in the raw content
    try:
        content = content.encode().decode('unicode_escape')
    except:
        pass  # If decoding fails, use original content
    
    json_objects = []
    lines = content.split('\n')
    current_object = ""
    brace_count = 0
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
            
        current_object += line + " "
        brace_count += line.count('{') - line.count('}')
        
        if brace_count == 0 and current_object.strip():
            try:
                clean_json = re.sub(r'\s+', ' ', current_object.strip())
                obj = json.loads(clean_json)
                
                # Recursively decode Unicode in the parsed object
                def decode_in_object(item):
                    if isinstance(item, dict):
                        return {k: decode_in_object(v) for k, v in item.items()}
                    elif isinstance(item, list):
                        return [decode_in_object(i) for i in item]
                    elif isinstance(item, str):
                        # Replace common Unicode escapes
                        return item.replace('\\u2026', '…').replace('\\u2019', "'").replace('\\u201c', '"').replace('\\u201d', '"')
                    return item
                
                obj = decode_in_object(obj)
                json_objects.append(obj)
                
            except json.JSONDecodeError as e:
                print(f"Failed to parse: {current_object[:100]}...")
                print(f"Error: {e}")
            current_object = ""
    
    # Write with Unicode characters preserved
    with open(output_file, "w", encoding='utf-8') as f:
        for obj in json_objects:
            f.write(json.dumps(obj, separators=(',', ':'), ensure_ascii=False) + "\n")
    
    return len(json_objects)

# Fix with Unicode decoding
count = fix_jsonl_with_unicode_decoding("yoda_chat_01.jsonl", "yoda_chat_02.jsonl")
print(f"Processed {count} objects with Unicode decoding")

# Check a few entries to see if Unicode is fixed
with open("yoda_chat_final.jsonl", "r") as f:
    for i, line in enumerate(f):
        if i < 3:  # Check first 3 entries
            obj = json.loads(line)
            print(f"Entry {i+1}:", obj)
            print()

Failed to parse: {"messages:" [{"role": "user", "content": "Is revenge a good thing"}, {"role": "assistant", "content...
Error: Expecting ':' delimiter: line 1 column 14 (char 13)
Processed 404 objects with Unicode decoding
Entry 1: {'messages': [{'role': 'user', 'content': 'Impossible! The Sith have been extinct for a millennium.'}, {'role': 'assistant', 'content': 'The very Republic is threatened, if involved the Sith are.'}]}

Entry 2: {'messages': [{'role': 'user', 'content': 'I do not believe they could have returned without us knowing.'}, {'role': 'assistant', 'content': 'Hard to see, the dark side is. Discover who this assassin is, we must.'}]}

Entry 3: {'messages': [{'role': 'user', 'content': 'This attack was with purpose, that is clear, and I agree the Queen is the target.'}, {'role': 'assistant', 'content': 'With this Naboo queen you must stay, Qui-Gon. Protect her.'}]}



In [15]:
import re
with open("yoda_chat_02.jsonl", "r") as f:
    content = f.read()
# Find all Unicode escape sequences
unicode_escapes = re.findall(r'\\u[0-9a-fA-F]{4}', content)
unique_escapes = set(unicode_escapes)
print("Found Unicode escapes:", unique_escapes)

Found Unicode escapes: set()


In [17]:
import json

# Test loading your final file
file = []
with open("yoda_chat_02.jsonl", "r") as f:
    for line_num, line in enumerate(f, 1):
        line = line.strip()
        if not line:
            continue
        try:
            file.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Error on line {line_num}: {e}")
            break

print(f"Successfully loaded {len(file)} entries")
print(f"File size check: {len(file)} entries loaded")

# Show a sample entry to verify content looks good
if file:
    print("\nSample entry:")
    print(json.dumps(file[0], indent=2))

Successfully loaded 404 entries
File size check: 404 entries loaded

Sample entry:
{
  "messages": [
    {
      "role": "user",
      "content": "Impossible! The Sith have been extinct for a millennium."
    },
    {
      "role": "assistant",
      "content": "The very Republic is threatened, if involved the Sith are."
    }
  ]
}


In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes

In [ ]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

In [ ]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/mistral-7b-instruct-v0.3"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

In [ ]:
from datasets import Dataset

def format_prompt(example):
    messages = {m["role"]: m["content"] for m in example["messages"]}
    user = messages.get("user", "").strip()
    yoda = messages.get("assistant", "").strip()
    return f"[INST] {user} [/INST]\n{yoda}"

# Apply to all examples and wrap in dict
dataset = [{"text": format_prompt(ex)} for ex in file]

dataset = Dataset.from_list(dataset)

print(dataset[0])  # see the first formatted sample



In [ ]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none", # Disable Weights & Biases logging
    ),
)

ModuleNotFoundError: No module named 'trl'

In [ ]:
# Train the model
trainer_stats = trainer.train()

In [ ]:
# Test the fine-tuned model
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Test prompt
messages = [
    {"role": "user", "content": "Tell me about the Force"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

# Generate response
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

# Decode and print
response = tokenizer.batch_decode(outputs)[0]
print(response)

NameError: name 'FastLanguageModel' is not defined

In [ ]:
# This model generates an error due to package conflicts. Package causing the error: entencepiece_model.proto
# model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")

# Load your trained weights
# model.load_adapter("yoda_model")  # This loads your adapter weights

# Instead run this new code
model.save_pretrained_merged("yoda_model_merged", tokenizer, save_method="merged_16bit", push_to_hub=False, max_shard_size="2GB")


NameError: name 'model' is not defined

In [ ]:
# FIRST: Save your model in HuggingFace format (this almost always works)
model.save_pretrained("yoda_model")
tokenizer.save_pretrained("yoda_model")

# THEN: Try the GGUF conversion
# If it fails, you still have your model saved!
try:
    model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")
except Exception as e:
    print(f"GGUF conversion failed: {e}")
    print("But your model is safely saved in HuggingFace format!")

NameError: name 'model' is not defined

In [ ]:
# This is to download the yoda model in hugging face format. I can even use this on my local machine later.

# # Zip the folder first for easier download
# import shutil
# shutil.make_archive('yoda_model', 'zip', 'yoda_model')

# # Then download the zip file
# from google.colab import files
# files.download('yoda_model.zip')

In [ ]:
# from google.colab import files
# import os

# gguf_files = [f for f in os.listdir("gguf_model") if f.endswith(".gguf")]
# if gguf_files:
#     gguf_file = os.path.join("gguf_model", gguf_files[0])
#     print(f"Downloading: {gguf_file}")
#     files.download(gguf_file)